# 🦜🔗 Langchain Demo

Hallo und herzlich Willkommen! 

## Umgebungsvariablen gesetzt?

```
# OpenAI Variante
OPENAI_API_KEY=<required>
OPENAI_MODEL="gpt-4-0125-preview"

# AzureVariante
AZURE_OPENAI_API_KEY=<required>
AZURE_OPENAI_ENDPOINT=<required>
AZURE_OPENAI_API_VERSION="2023-12-01-preview"
AZURE_OPENAI_DEPLOYMENT_NAME=<required>
AZURE_OPENAI_EMBEDDING_NAME=<required>
```

Diese Umgebungsvariablen werden implizit verwendet, wenn man ein LLM aufruft oder ein Textstück vektorisiert.

In [ ]:
%pip install -r requirements.txt

In [ ]:
from dotenv import load_dotenv
load_dotenv()

## Woher der Hype?

2017 kam ein Paper von Google heraus: [Attention is all you need](https://arxiv.org/abs/1706.03762). Die dort vorgestellte Transformer-Architektur unterscheidet sich von den bis dahin gebräuchlichen neuronalen Netzen. Das Netz wird insbesondere darauf trainiert, gezielt die "Aufmerksamkeit" auf "relevante" Passagen eines möglicherweise sehr langen Inputs zu lenken. Das macht es ideal zur Verarbeitung von viel Text. Das Paper wurde bisher überaus oft zitiert. Wer keine Angst vor Technikt hat und sich dafür interessiert, was eigentlich hinter so einem LLM steckt, könnte bei diesem Paper mit der Reise beginnen.

## Überblick

### OpenAI-Modelle
ChatGPT oder auch jedes andere LLM benutzen ist relativ einfach mit Langchain

In diesen Test nutzen wir das neueste "gpt-4-turbo" Model - mögliche Large Language Modelle von OpenAI sind:
- `gpt-35-turbo`  Das günstigste und am weitesten verbreitete Modell
- `gpt-4`  Das neue und bessere GPT Modell
- `gpt-4-turbo`  Turbo-variante von gpt-4 (günstiger, schneller, kleinere maximale Länge des Text-Outputs)
- `gpt-4-vision`  Ein "multimodales" Modell, welches auch auf Bilder trainiert wurde.

OpenAI trainiert diese Versionen laufend neu, was dazu führen kann, das Anfragen an das LLM plötzlich andere Antworten geben.
Möchte man dies verhindern, kann man seine Applikation auf einen Snapshot (z.b. gpt-4-0613) festsetzen.
Dies ist insbesondere wichtig, wenn die Applikation vom Output des LLM bestimmte Strukturen erwartet, beispielsweise eine bestimmte XML-Syntax o.Ä.

OpenAI-Modelle werden nicht nur von OpenAI selbst gehostet, sondern auch von Azure.
Diese muss man auf dem Azure Portal selbst als Endpunkte konfigurieren, in der Regel leiden die OpenAI Azure Deployments weniger unter hoher Auslastung

### Andere Modelle
Auch wenn wir nicht damit arbeiten werden, ist es vielleicht ganz gut, die Namen der "großen" Konkurrenz-Modelle einmal gehört zu haben:
- `Gemini` Das neueste Google-Modell. Es hat den Fokus insbesondere auf multimodalem Input.
- `Claude` Claude ist die LLM-Reihe von Anthropic. Sehr viel Instruction-Tuning.
- `Mixtral` Ein sehr gutes Open-Source Modell. Entwickelt von Mistral AI. Ein guter Kandidat für ein selbst gehostetes LLM.

### Temperatur
Alle LLMs sind nicht deterministisch. Aber die Temperatur ist ein Parameter, mit der man die Variabilität von Antworten hoch und runter schrauben kann.
Wie bei normalen Atomen ist die Bewegung niedrig, wenn die Temperatur niedrig ist. Wenn man die Temperatur hochschraubt, wird viel gewackelt.
Der Temperatur-Parameter ist üblicherweise ein Fließkommawert zwischen 0 und 1.

### Streaming
Nicht alle LLMs bieten die Möglichkeit, Token für Token live zu streamen. OpenAI-Modelle können es, man kann dies mit dem Streaming-Parameter einstellen.


### Bitte etwas schneller
Recht neu auf dem Markt mit beeindruckendem Token/Sekunde-Verhältnis: [Groq](https://groq.com/)

### Links:
- https://python.langchain.com/docs/get_started/introduction
- https://platform.openai.com/docs/models/

#### Wir probieren aus:

In [ ]:
from helpers import llm
print(llm().invoke("Hi OpenAI! Kannst Du mir gerade mal einen fränkischen Trinkspruch auf die Stadt Würzburg im Dialekt erzeugen?").content)

#### Jetzt nochmal mit Streaming. Dazu rufen wir nicht invoke sondern astream auf (a für async). Wir drehen etwas an der Temperatur, damit die Ergebnisse spannend bleiben

In [ ]:
chunks = []
async for chunk in llm(temperature=1).astream("Erklär mir in einem Satz Quantenmechanik."):
    chunks.append(chunk)
    print(chunk.content, end="", flush=True)

## Token

Token sind die kleinste Einheit des LLM. Das haben wir gerade beim Streaming schön gesehen. Der Stream kommt Token für Token aus dem LLM gepurzelt.

Das LLM rechnet aus der Eingabe und den bisher errechneten Token die Wahrscheinlichkeit für den nächsten Token aus. Dieser neue Token wird dann angehängt und der nächste Token wird ermittelt.

So geht das immer weiter. Bis der nächste wahrscheinlichste Token ein Stop-Zeichen ist. Auf diese Weise generieren LLMs die wahrscheinlichste Fortführung der Eingabetoken.

Token können Wörter, machmal sogar Wortgruppen oder auch nur einzelne oder mehrere Buchstaben sein.

Die Bepreisung der LLMs ist an die Tokenanzahl (Eingabe und Ausgabe) gekoppelt.



Links:
- https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb 

In [ ]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

tokens = encoding.encode("AI ist eine tolle Sache.")

decoded_tokens = [encoding.decode_single_token_bytes(token).decode('utf-8') for token in tokens]
for token in decoded_tokens:
    print(token)

## Prompt Engineering und Templates in Langchain

Um die Dinge von der AI zu bekommen, die man erwartet, stellt man am besten sehr konkrete und präzise Anfragen.

Weil eine AI oft an ein bestimmtes Feld von Aufgaben gekoppelt ist, gibt man die Rahmenanweisung dann in ein Template ein, um nicht immer wieder die gleiche Rahmenanweisung zu schreiben.

Die jeweilige konkrete Nutzeranfrage wird dann in das Template eingefügt und das ausgefüllte Template ans LLM übergeben.

Der Trend geht immer mehr zu Chat-Modellen. Hierbei ist die Information, die man dem LLM gibt, in "Messages" unterteilt. Besondere Gewichtung hat eine System-Message. Diese kann Rahmenanweisungen enthalten, an die sich das LLM halten soll. Dem Nutzer wird es schwer fallen, das LLM dazu zu bewegen, sich über eine Anweisung in der System-Message hinweg zu setzen. Das LLM wurde ganz einfach darauf trainiert, sich an die Anweisungen einer System-Message strikt zu halten.

### Links
- https://python.langchain.com/docs/get_started/quickstart#prompt-templates
- https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/prompt-engineering
- https://learnprompting.org/docs/intro
- https://www.promptingguide.ai/
- https://smith.langchain.com/hub

In [ ]:
from langchain.prompts import  ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Du bist ein {beruf} aus Würzburg."),
        ("human", "Erkläre in 2 Sätzen im fränkischen dialekt warum Deine Kunden aus {ort} die besten sind."),
    ]
)

print(prompt.format(beruf="Winzer", ort="Würzburg"))


### Langchain Hub Beispiel

Weil das "Prompt-Engineering" ein bisschen Übung braucht und es diverse Tricks gibt, hat LangChain einen "Hub", auf dem man eine ganze Reihe vorgefertigter Prompts für verschiedene Anwendungsfälle findet.

Dort kann man sich inspirieren lassen, Prompts forken oder auch selbst etwas für andere Leute zur Verfügung stellen, wenn es sich als nützlich erweist.

Links:
- https://smith.langchain.com/hub/borislove/customer-sentiment-analysis

In [ ]:
from langchain import hub
sentiment_prompt = hub.pull("borislove/customer-sentiment-analysis")

client_letter="""Ich bin von dem Volleyballschläger zutiefst enttäuscht. Zuerst ist der Griff abgefallen, danach auch noch der Dynamo. Außerdem riecht er noch schlechter als er schmeckt. Wieso ist das immer so ein Ärger mit euch?"""
format_instructions="""Zusätlich zur numerischen Klassifizierung sollst du herausfinden, was der Kunde gerne gehabt hätte. Antworte auf deutsch."""

print(sentiment_prompt.format(client_letter = client_letter, format_instructions = format_instructions))

## Jetzt fängt es an, etwas technischer zu werden. Wieso heißt LangChain eigentlich LangChain?

Langchain definiert einige Python-Operatoren neu, wenn sie zwischen LangChain-Objekten stehen. Der bekannteste ist die Pipe: |

Wenn die Pipe zwischen zwei Langchain-Objekten steht, wird die Ausgabe des ersten Obekts an das nächste weitergegeben. Damit erhält man eine "Chain" von "Runnables"

#### Links
- https://python.langchain.com/docs/modules/chains

In [ ]:
from langchain.schema import StrOutputParser # Hilft beim Formatieren
chain = prompt | llm() | StrOutputParser()
print(chain.invoke({"beruf":"Winzer", "ort":"Würzburg"}))

In [ ]:
# Streaming
async for chunk in chain.astream({"beruf":"Brauer", "ort":"Würzburg"}):
    print(chunk, end="", flush=True)

In [ ]:
# Funktioniert auch das Beispiel vom Hub?
sentiment_chain = sentiment_prompt | llm() | StrOutputParser()
async for chunk in sentiment_chain.astream({"client_letter" :client_letter, "format_instructions" : format_instructions}):
    print(chunk, end="", flush=True)

In [ ]:
# Wir können dynamisch die format_instructions des Templates überschreiben, um neue Ergebnisse zu bekommen
format_instructions="""Zusätlich zur sentiment Analysis ist es deine Aufgabe, die Sinnhaftigkeit der Kundenäußerung zu überprüfen."""
async for chunk in sentiment_chain.astream({"client_letter" :client_letter, "format_instructions" : format_instructions}):
    print(chunk, end="", flush=True)

### Debug Informationen gewünscht?

In [ ]:
from langchain.globals import set_debug

In [ ]:
## Und jetzt selber mal Ausprobieren
set_debug(True)
print(chain.invoke({
    'beruf': "Programmierer",
    'ort': "Würzburg"
    }))